##### 提示技巧

- 图像放置在文本之前效果最佳
- 多图像时，用 "Image 1:"、"Image 2:" 等介绍每张图片

安装一个显示加载图像的库

In [ ]:
%pip install IPython requests Pillow

In [ ]:
message_list = [
    {
        "role": 'user',
        "content": [
            {"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": get_image_from_url("https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20240828144957.png")}},
            {"type": "text", "text": "按照流程图生成代码"}
        ]
    }
]

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=message_list
)
print(response.content[0].text)

In [ ]:

message = client.messages.create(
    model = MODEL_NAME,
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "图像 1："
                },
                {
                    "type": "image",
                    "source":  {"type": "base64", "media_type": "image/jpeg", "data": get_image_from_file("images/spot_diff1.jpg")},
                },
                {
                    "type": "text",
                    "text": "图像 2："
                },
                {
                    "type": "image",
                    "source":  {"type": "base64", "media_type": "image/jpeg", "data": get_image_from_file("images/spot_diff2.jpg")},
                },
                {
                    "type": "text",
                    "text": "这些图像有什么不同？仔细的找不同"
                }
            ],
        }
    ],
)


In [ ]:
生成具有不同细节级别的图片和视频说明。我们建议您从以下图片/视频提示开始，从此处迭代操作以获取更具体的说明。
图片：“您能编写关于图片的说明吗？”
视频：“您能编写这个视频所发生情况的说明吗？”

对象边界框
询问图像中对象的边界框，增强目标检测能力。

In [ ]:
bbox_prompt = "Return a bounding box for the piranha. \n [ymin, xmin, ymax, xmax]"
bbox_response = generate_content(api_key, uploaded_image['uri'], bbox_prompt)
print("Bounding Box:", bbox_response['text'])

def convert_coordinates(coordinates, original_size):
    ymin, xmin, ymax, xmax = map(float, coordinates.split(','))
    height, width = original_size
    return [ymin * height, xmin * width, ymax * height, xmax * width]

音频

In [ ]:
your_file = genai.upload_file(path='sample.mp3')

In [ ]:
model.count_tokens([your_file])

In [ ]:

prompt = "Listen carefully to the following audio file. Provide a brief summary."
model = genai.GenerativeModel('models/gemini-1.5-flash')
response = model.generate_content([prompt, your_file])
print(response.text)

对于小请求，您可以将音频数据内联到请求中，就像处理图像一样。使用 PyDub 修剪音频的前 10 秒：

In [ ]:
!pip install -Uq pydub

In [ ]:
from pydub import AudioSegment

In [ ]:
sound = AudioSegment.from_mp3("sample.mp3")
sound[:10000] # slices are in ms

In [ ]:

response = model.generate_content([
    "Please transcribe this recording:",
    {
        "mime_type": "audio/mp3",
        "data": sound[:10000].export().read()
    }
])
from IPython import display

display.Markdown(response.text)

视频处理能力概述

Gemini 1.5 Pro和Flash能够处理长达一小时的视频数据。这为分析较长视频内容提供了很大的灵活性。

支持的视频格式

这两个系统支持多种常见的视频格式,具体包括:

1. MP4 (video/mp4)
2. MPEG (video/mpeg) 
3. MOV (video/mov)
4. AVI (video/avi)
5. FLV (video/x-flv)
6. MPG (video/mpg)
7. WebM (video/webm)
8. WMV (video/wmv)
9. 3GPP (video/3gpp)

这种多格式支持意味着用户可以使用大多数常见的视频文件,无需进行格式转换。

视频处理方式

系统会以特定的方式处理视频内容:

- 视频帧提取: 每秒提取1帧图像(1 FPS)
- 音频处理: 以1Kbps的速率提取单通道音频
- 时间戳添加: 每秒为提取的内容添加时间戳

需要注意的是,1 FPS的采样率可能会导致快速动作场景的细节丢失。如果视频中包含高速移动的重要内容,建议将这些片段放慢速度,以提高分析质量。

数据处理能力

系统对视频内容的处理量化如下:

- 每个视频帧包含258个词元
- 音频每秒产生32个词元
- 算上元数据,视频每秒大约生成300个词元
- 总计可处理约100万个词元,相当于接近一小时的视频长度

使用技巧

1. 时间戳查询: 如需询问视频中特定时间点的内容,请使用"MM:SS"格式,例如"05:30"表示5分30秒。

2. 单一视频原则: 每次分析最好只使用一个视频,避免多视频混淆。

3. 提示位置: 如果只分析一个视频,请将文字提示放在视频信息之后。

In [ ]:

system_prompt = """
You are historian who specializes in events caught on film.
When you receive a video answer following questions:
When did it happen?
Who is the most important person in video?
How the event is called?
"""

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]
 model = genai.GenerativeModel(model_name="models/gemini-1.5-flash", safety_settings=safety_settings,
                              system_instruction=system_prompt)
response = model.generate_content([video_file])
print(response.text)

In [ ]:

# Upload video
video_file = genai.upload_file(path=path)

In [ ]:

# 视频上传后，状态为 PROCESSING，需等其变为 ACTIVE 才可用于推理。
while video_file.state.name == "PROCESSING":
  print('.', end='')
  time.sleep(5)
  video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

In [ ]:
# Display some of the video content
cap = cv2.VideoCapture(path)
frame_number = 1000
for _ in range(frame_number):
    ret, frame = cap.read()
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

plt.imshow(frame_rgb)
plt.axis('off')
plt.show()

cap.release()

In [ ]:
system_prompt = "You should provide a quick 2 or 3 sentence summary of what is happening in the video."

In [ ]:
model = genai.GenerativeModel(model_name="models/gemini-1.5-flash", system_instruction=system_prompt)
response = model.generate_content([video_file])
print(response.text)

In [ ]:
transcript_prompt = "Transcribe the audio, giving timestamps. Also provide visual descriptions."
transcript_response = generate_content(api_key, video_file['uri'], transcript_prompt)
print("Transcription Output:", transcript_response['text'])

In [ ]:
Gemini API 每秒仅处理一帧视频。这可能导致模型无法看到发生的所有事情，特别是当某些东西仅在一瞬间可见时。

综合传入

In [ ]:
### 目标
将原始思想和创意转化为引人入胜的博客文章，展现作者独特的风格和声音。

### 输入
1. **示例博客文章（1-5）**：
   - 用户提供与期望风格相符的博客文章示例，帮助理解用词、句子结构和整体声音。

2. **音频剪辑**：
   - 用户分享脑风暴的音频录音，内容如同与朋友讨论，表达自由开放。

### 输出
- **博客文章草稿**：
  - 一份结构清晰的初稿，适合在 Substack 或 LinkedIn 等平台发布，内容包括：
  - 清晰而引人入胜的写作，确保对目标受众有趣。
  - 语调和风格与示例一致，确保一致性。
  - 逻辑流畅的结构，分为明确的章节。
  - 目标字数为 500-800 字，可根据用户偏好调整。

### 过程
1. **风格分析**：
   - 分析示例博客，识别以下元素：
     - **词汇与用词**：正式/非正式、专业术语/俚语。
     - **句子结构与长度**：短句/长句。
     - **语调与声音**：幽默、严肃、信息性、说服性等。

2. **音频转录与理解**：
   - 高精度转录音频，提取关键想法、论点和支持点。

3. **草稿生成**：
   - 根据音频见解和风格指南生成初稿，确保包含所有相关章节及支持论据，结尾精彩，吸引读者继续关注。


In [ ]:
prompt = "Draft my next blog post based on my thoughts in this audio file and these two previous blog posts I wrote."

model = genai.GenerativeModel(model_name="models/gemini-1.5-flash", system_instruction=si)

response = model.generate_content([prompt, blog_file, blog_file2, audio_file],
                                  request_options={"timeout": 600})